In [1]:
# ==============================================================================
#  Augmented Training Script for Plant Disease Identification
#  A lot of the comments in this code is written by AI to make it as verbally clear as possible
#  This script will fine-tune the original Hugging Face model using
#  data augmentation to create a more robust and accurate model that
#  is better at generalizing to new, unseen images.
# ==============================================================================

# --- Core Installations ---
!pip install -q --upgrade pip
!pip install -q transformers huggingface_hub pillow matplotlib opencv-python-headless tqdm

# --- Core Imports ---
import sys, os, zipfile, warnings
from google.colab import files
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from transformers import AutoModelForImageClassification
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
print("Python", sys.version)

# Enhanced Configuration
# Set to TRAINING mode
MODE = 'TRAINING'

# Start from the original, pre-trained model from Hugging Face
HF_MODEL_ID = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"SCRIPT MODE: {MODE}")
print("Using device:", DEVICE)

# Load the Base Model
print(f"Loading base model '{HF_MODEL_ID}' from Hugging Face...")
try:
    model = AutoModelForImageClassification.from_pretrained(HF_MODEL_ID).to(DEVICE)
    print("Base model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}\nPlease check internet connection and model ID.")
    exit()




# MAIN SCRIPT LOGIC



if MODE == 'TRAINING':
    # TRAINING PIPELINE WITH DATA AUGMENTATION
    print("\n--- AUGMENTED TRAINING MODE ---")

    # 1. UPLOAD AND UNZIP DATASET
    print("\nSTEP 1: Please upload your dataset as a .zip file (e.g., archive.zip).")
    uploaded = files.upload()

    if not uploaded:
        print("No dataset uploaded. Please re-run to upload a dataset.")
    else:
        zip_file_name = list(uploaded.keys())[0]

        if not zipfile.is_zipfile(zip_file_name):
            print(f"❌ Error: Uploaded file '{zip_file_name}' is not a valid zip file.")
        else:
            print(f"\nUnzipping {zip_file_name}...")
            !unzip -q "{zip_file_name}"
            print("Unzipping complete.")

            dataset_path = "./PlantVillage"

            # 2. DEFINE DATA AUGMENTATION AND LOADING
            # Define a set of random transformations for data augmentation
            train_transforms = T.Compose([
                T.RandomResizedCrop(size=224, scale=(0.8, 1.0)), # Randomly zoom and crop
                T.RandomHorizontalFlip(),                        # Flip the image horizontally (50% chance)
                T.RandomRotation(degrees=20),                    # Rotate the image up to 20 degrees
                T.ColorJitter(brightness=0.2, contrast=0.2),     # Randomly change brightness and contrast
                T.ToTensor(),                                    # Convert the PIL Image to a PyTorch Tensor
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with standard values
            ])

            # A simpler transform for the validation set (no random changes, just resize and normalize)
            val_transforms = T.Compose([
                T.Resize(256),
                T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

            try:
                # Create separate datasets for training and validation with their respective transforms
                train_dataset_full = ImageFolder(root=dataset_path, transform=train_transforms)
                val_dataset_full = ImageFolder(root=dataset_path, transform=val_transforms)

                # Split the data into training and validation sets
                train_size = int(0.8 * len(train_dataset_full))
                val_size = len(train_dataset_full) - train_size
                # Use torch.utils.data.random_split to ensure the same split is used for both datasets
                indices = torch.randperm(len(train_dataset_full)).tolist()
                train_dataset = torch.utils.data.Subset(train_dataset_full, indices[:train_size])
                val_dataset = torch.utils.data.Subset(val_dataset_full, indices[train_size:])

                # The collate_fn is no longer needed because transforms handle everything.
                # PyTorch's default collate function works perfectly now.
                train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
                val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

                print(f"\nDataset loaded: {len(train_dataset_full)} images from {len(train_dataset_full.classes)} classes.")
                print(f"Training set: {len(train_dataset)} images | Validation set: {len(val_dataset)} images")

                # 3. SETUP FOR TRAINING
                model.train() # Set the model to training mode
                optimizer = optim.AdamW(model.parameters(), lr=5e-5)
                criterion = nn.CrossEntropyLoss()
                num_epochs = 5 # Increased epochs for better learning with augmentation

                # 4. TRAINING & VALIDATION LOOP
                print("\nSTEP 2: Starting augmented training...")
                for epoch in range(num_epochs):
                    # --- Training Phase ---
                    model.train()
                    running_loss, correct_predictions, total_samples = 0.0, 0, 0
                    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} [Training]"):
                        images, labels = images.to(DEVICE), labels.to(DEVICE)

                        outputs = model(images)
                        loss = criterion(outputs.logits, labels)

                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                        running_loss += loss.item() * labels.size(0)
                        _, predicted = torch.max(outputs.logits, 1)
                        correct_predictions += (predicted == labels).sum().item()
                        total_samples += labels.size(0)
                    epoch_train_loss = running_loss / total_samples
                    epoch_train_acc = correct_predictions / total_samples

                    # --- Validation Phase ---
                    model.eval()
                    val_loss, val_correct, val_total = 0.0, 0, 0
                    with torch.no_grad():
                        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} [Validation]"):
                            images, labels = images.to(DEVICE), labels.to(DEVICE)

                            outputs = model(images)
                            loss = criterion(outputs.logits, labels)

                            val_loss += loss.item() * labels.size(0)
                            _, predicted = torch.max(outputs.logits, 1)
                            val_correct += (predicted == labels).sum().item()
                            val_total += labels.size(0)
                    epoch_val_loss = val_loss / val_total
                    epoch_val_acc = val_correct / val_total

                    # --- EPOCH LOG ---
                    print(f"Epoch {epoch + 1}/{num_epochs} | "
                          f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc*100:.2f}% | "
                          f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc*100:.2f}%")

                print("\nFinished Augmented Training!")
                print("You can now save your new, more robust model using:")
                print("model.save_pretrained('./my_augmented_model')")

            except FileNotFoundError:
                print(f"❌ Error: Could not find dataset directory '{dataset_path}'.")
                print("Please double-check the 'dataset_path' variable and your zip file's structure.")
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}")
else:
    print(f"Invalid MODE: '{MODE}'. This script is configured for TRAINING.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00
Python 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
SCRIPT MODE: TRAINING
Using device: cpu
Loading base model 'linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification' from Hugging Face...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/9.34M [00:00<?, ?B/s]

Base model loaded successfully.

--- AUGMENTED TRAINING MODE ---

STEP 1: Please upload your dataset as a .zip file (e.g., archive.zip).


model.safetensors:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

...

Epoch 4/5 [Validation]:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch 4/5 | Train Loss: 0.1037, Train Acc: 96.85% | Val Loss: 0.1992, Val Acc: 94.38%


Epoch 5/5 [Training]:   0%|          | 0/516 [00:00<?, ?it/s]

Epoch 5/5 [Validation]:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch 5/5 | Train Loss: 0.0810, Train Acc: 97.65% | Val Loss: 0.0582, Val Acc: 98.43%

Finished Augmented Training!
You can now save your new, more robust model using:
model.save_pretrained('./my_augmented_model')


In [2]:
model.save_pretrained('./my_augmented_model')

In [3]:
!zip -r my_augmented_model.zip ./my_augmented_model

  adding: my_augmented_model/ (stored 0%)
  adding: my_augmented_model/model.safetensors (deflated 7%)
  adding: my_augmented_model/config.json (deflated 69%)
